In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Activation
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [3]:
# whether to use LSTM or MLP
use_LSTM = True

# number of features used in the regression (for MLP)
mlp_num_features = 5
#

# predict several timesteps at once
lstm_predict_sequences = True
lstm_num_predictions = 5

# lstm_num_timesteps
lstm_num_timesteps = 5
# lstm_num_features
lstm_num_features = 1
# stateful?
lstm_stateful = True
# use two lstm layers?
lstm_stack_layers = False

# window_size
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features

batch_size = 1
num_epochs = 200
# dimensionality of the output space
num_neurons = 4

# scale the dataset to values between scale_min and scale_max
scale = False
scale_min = -1
scale_max = 1
#scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
scaler = StandardScaler()

# number of consecutive (dependent) predictions
prediction_window = 5 


In [4]:
testname = 'predict'
properties = 'lstm_' + str(use_LSTM) + '_stateful_' + str(lstm_stateful) + '_window_' + str(window_size) + '_predict_sequences_' + str(lstm_predict_sequences) + '_epochs_' + str(num_epochs) + '_2layers_' + str(lstm_stack_layers) + '_scale_' + str(scale)
model_name = testname + '_' + properties + '.h5'
fig_name = testname + '_' + properties + '.png'
fig_name_multiple = testname + '_' + properties + '_multiplepreds_' + str(prediction_window) + '.png'


model_exists = False

In [5]:
filename = '108_7_3.train.csv'
df_train = pd.read_csv(filename, usecols=[1])
ts_train = df_train.values.astype('float64')
df_test = pd.read_csv(filename, usecols=[1])
ts_test = df_test.values.astype('float64')
ts_all = np.append(ts_train, ts_test).reshape(-1,1)
len_overall = len(ts_all)

In [6]:
len_overall

1798

In [7]:
ts_train.shape, ts_test.shape

((899, 1), (899, 1))

In [8]:
ts_train[:10]

array([[ 4164.],
       [ 3540.],
       [ 3560.],
       [ 3094.],
       [ 3658.],
       [ 3049.],
       [ 3526.],
       [ 3290.],
       [ 4447.],
       [ 2305.]])

In [9]:
ts_test[:10]

array([[ 4164.],
       [ 3540.],
       [ 3560.],
       [ 3094.],
       [ 3658.],
       [ 3049.],
       [ 3526.],
       [ 3290.],
       [ 4447.],
       [ 2305.]])

In [10]:
if scale:
    ts_train = scaler.fit_transform(ts_train)
    ts_test = scaler.transform(ts_test)

In [11]:
ts_train[:10]

array([[ 4164.],
       [ 3540.],
       [ 3560.],
       [ 3094.],
       [ 3658.],
       [ 3049.],
       [ 3526.],
       [ 3290.],
       [ 4447.],
       [ 2305.]])

In [12]:
ts_test[:10]

array([[ 4164.],
       [ 3540.],
       [ 3560.],
       [ 3094.],
       [ 3658.],
       [ 3049.],
       [ 3526.],
       [ 3290.],
       [ 4447.],
       [ 2305.]])

In [13]:
# split into train and test sets
#train_size = int(len(ts) * 0.67)
#test_size = len(ts) - train_size
#ts_train, ts_test = ts[0:train_size,:], ts[train_size:len(ts),:]
#print(len(ts_train), len(ts_test))


In [14]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size), 0]
        dataX.append(a)
        dataY.append(dataset[i + window_size, 0])
    return np.array(dataX), np.array(dataY)

In [15]:
def create_dataset_multiple_y(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

In [16]:
if use_LSTM:
    if lstm_predict_sequences:
        X_train, y_train = create_dataset_multiple_y(ts_train, lstm_num_timesteps)
        X_test, y_test = create_dataset_multiple_y(ts_test, lstm_num_timesteps)
    else:    
        X_train, y_train = create_dataset(ts_train, lstm_num_timesteps)
        X_test, y_test = create_dataset(ts_test, lstm_num_timesteps)
else:
    X_train, y_train = create_dataset(ts_train, mlp_num_features)
    X_test, y_test = create_dataset(ts_test, mlp_num_features)
    
# the train and test matrices end up shorter than the respective timeseries by window_size + 1!
X_train.shape, y_train.shape, X_test.shape,  y_test.shape

((889, 5), (889, 5), (889, 5), (889, 5))

In [17]:
X_train[:5,:]

array([[ 4164.,  3540.,  3560.,  3094.,  3658.],
       [ 3540.,  3560.,  3094.,  3658.,  3049.],
       [ 3560.,  3094.,  3658.,  3049.,  3526.],
       [ 3094.,  3658.,  3049.,  3526.,  3290.],
       [ 3658.,  3049.,  3526.,  3290.,  4447.]])

In [18]:
y_train[:5]

array([[ 3049.,  3526.,  3290.,  4447.,  2305.],
       [ 3526.,  3290.,  4447.,  2305.,  2657.],
       [ 3290.,  4447.,  2305.,  2657.,  2784.],
       [ 4447.,  2305.,  2657.,  2784.,  3068.],
       [ 2305.,  2657.,  2784.,  3068.,  4036.]])

In [19]:
X_test[:5,:]

array([[ 4164.,  3540.,  3560.,  3094.,  3658.],
       [ 3540.,  3560.,  3094.,  3658.,  3049.],
       [ 3560.,  3094.,  3658.,  3049.,  3526.],
       [ 3094.,  3658.,  3049.,  3526.,  3290.],
       [ 3658.,  3049.,  3526.,  3290.,  4447.]])

In [20]:
y_test[:5]

array([[ 3049.,  3526.,  3290.,  4447.,  2305.],
       [ 3526.,  3290.,  4447.,  2305.,  2657.],
       [ 3290.,  4447.,  2305.,  2657.,  2784.],
       [ 4447.,  2305.,  2657.,  2784.,  3068.],
       [ 2305.,  2657.,  2784.,  3068.,  4036.]])

In [21]:
if use_LSTM:
    # reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    
    if lstm_predict_sequences:
        y_train = np.reshape(y_train, (y_train.shape[0], lstm_num_predictions, lstm_num_features))
        y_test = np.reshape(y_test, (y_test.shape[0], lstm_num_predictions, lstm_num_features))

In [22]:
if not model_exists:
    
    model = Sequential()

    # LSTM input shape
    # (samples, time steps, features)
    # LSTM output shape
    # if return_sequences: 3D tensor with shape (batch_size, timesteps, units).
    # else, 2D tensor with shape (batch_size, units).

    if use_LSTM:

        print('LSTM')
        # the last state for each sample at index i in a batch will be used as initial state
        # for the sample of index i in the following batch
        if lstm_stateful:
            print('stateful')
            #
            if lstm_stack_layers:
                print('stack_layers')
                model.add(LSTM(num_neurons,
                           batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                           stateful = True,
                           return_sequences = True))
                print(model.output_shape)
                model.add(LSTM(num_neurons,
                           stateful = True))
                print(model.output_shape)
                model.add(Dense(1))
                print(model.output_shape)
                model.compile(loss='mean_squared_error', optimizer='adam')

            # 
            elif lstm_predict_sequences:
                print('predict_sequences')
                model.add(LSTM(num_neurons,
                           batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                           stateful = True,
                           return_sequences = True))
                print(model.output_shape)
                model.add(TimeDistributed(Dense(1)))
                print(model.output_shape)
                model.add(Activation("linear"))  
                model.compile(loss='mean_squared_error', optimizer='adam')

            #    
            else:
                print('predict single')
                model.add(LSTM(num_neurons,
                           batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                           stateful = True))
                print(model.output_shape)
                model.add(Dense(1))
                print(model.output_shape)
                model.compile(loss='mean_squared_error', optimizer='adam')



        # stateful == False    
        else: 
            print('stateless')

            if lstm_stack_layers:
                print('stack layers')
                # input_dim: dimensionality of the input (alternatively, input_shape)
                # required when using this layer as the first layer in a model
                model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True))
                print(model.output_shape)
                model.add(LSTM(num_neurons))
                print(model.output_shape)
                model.add(Dense(1))
                print(model.output_shape)
                model.compile(loss='mean_squared_error', optimizer='adam')
            # 
            # 
            elif lstm_predict_sequences:
                print('predict_sequences')
                model.add(LSTM(num_neurons,
                          #  input_dim = lstm_num_features,
                           batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]), 
                           return_sequences = True))
                print(model.output_shape) 
                model.add(TimeDistributed(Dense(1)))
                print(model.output_shape) 
                model.compile(loss='mean_squared_error', optimizer='adam')

            else:
                print('predict single')
                model.add(LSTM(num_neurons, 
                               input_shape=(X_train.shape[1], X_train.shape[2])
                               #input_dim = lstm_num_features
                              )
                              )
                print(model.output_shape) 
                model.add(Dense(1))
                print(model.output_shape) 
                model.compile(loss='mean_squared_error', optimizer='adam')



    # feedforward
    else:
        print('MLP')

        model.add(Dense(num_neurons, input_dim = mlp_num_features, activation='relu'))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')

else:
    print('loading from file: ' + model_name)
    model = load_model(model_name)

LSTM
stateful
predict_sequences
(1, 5, 4)
(1, 5, 1)


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 5, 4)                 96        
_________________________________________________________________
time_distributed_1 (TimeDist (1, 5, 1)                 5         
_________________________________________________________________
activation_1 (Activation)    (1, 5, 1)                 0         
Total params: 101.0
Trainable params: 101.0
Non-trainable params: 0.0
_________________________________________________________________


In [ ]:
if not model_exists:
    if use_LSTM & lstm_stateful:

        for i in range(num_epochs):
                print('epoch: ' + str(i))
                # shuffle must be False!
                model.fit(X_train, y_train, epochs = 1, batch_size = batch_size, shuffle = False)
                model.reset_states()

    else: 
        model.fit(X_train, y_train, epochs = num_epochs, batch_size = batch_size)


epoch: 0
Epoch 1/1
889/889 [==============================] - 2s - loss: 3632207.0458     
epoch: 1
Epoch 1/1
889/889 [==============================] - 2s - loss: 3626658.5357     
epoch: 2
Epoch 1/1
889/889 [==============================] - 2s - loss: 3620810.3312     
epoch: 3
Epoch 1/1
889/889 [==============================] - 2s - loss: 3614914.3772     
epoch: 4
Epoch 1/1
889/889 [==============================] - 2s - loss: 3609006.7882     
epoch: 5
Epoch 1/1
889/889 [==============================] - 3s - loss: 3603098.7289     
epoch: 6
Epoch 1/1
889/889 [==============================] - 3s - loss: 3597194.2579     
epoch: 7
Epoch 1/1
889/889 [==============================] - 3s - loss: 3591294.9859     
epoch: 8
Epoch 1/1
889/889 [==============================] - 3s - loss: 3585401.5304     
epoch: 9
Epoch 1/1
889/889 [==============================] - 3s - loss: 3579514.1121     
epoch: 10
Epoch 1/1
889/889 [==============================] - 2s - loss: 3573632.8843    

In [ ]:
if not model_exists:
    model.save(model_name)

In [ ]:
test_loss = np.nan
if lstm_stateful:
    test_loss = model.evaluate(X_test, y_test, batch_size = batch_size)
else:
    test_loss = model.evaluate(X_test, y_test, batch_size = X_test.shape[0])
test_loss

In [ ]:
if lstm_stateful:
    model.reset_states()
    pred_train = model.predict(X_train, batch_size = batch_size)
    model.reset_states()
    pred_test = model.predict(X_test, batch_size = batch_size)
else:
    pred_train = model.predict(X_train, batch_size = batch_size)
    pred_test = model.predict(X_test, batch_size = batch_size)

In [ ]:
for i in X_test:
    if use_LSTM:
        if lstm_stateful:
            model.reset_states()
        #print(i)
        r = i.reshape(1, len(i), 1)
        #print(i.shape), print(r.shape)
        print(model.predict(r))
    else:
        r = i.reshape(1, len(i))
        #print(i.shape), print(r.shape)
        print(model.predict(r))

In [ ]:
def calc_dependent_predictions(model, data, prediction_window):
    prediction_seqs = []
    for i in range(int(len(data)/prediction_window)):
        print('Calculating predictions starting from: {}'.format(i))
        curr_frame = data[i*prediction_window]
        predicted = []
        for j in range(prediction_window):
            #print('Calculating single prediction: {}'.format(j))
            #print(curr_frame)
            pred = model.predict(curr_frame[np.newaxis,:,:])[0,0]
            #pred = model.predict(curr_frame.reshape(1, len(curr_frame), 1)) # same
            #print(pred)
            predicted.append(pred)
            curr_frame = curr_frame[1:] 
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [ ]:
if use_LSTM and not lstm_predict_sequences:
    prediction_seqs_train = calc_dependent_predictions(model, X_train, prediction_window)
    prediction_seqs_test = calc_dependent_predictions(model, X_test, prediction_window)

In [ ]:
if use_LSTM and not lstm_predict_sequences:
    print(prediction_seqs_train)

In [ ]:
if use_LSTM and not lstm_predict_sequences:
    print(prediction_seqs_test)

In [ ]:
y_train[:10]

In [ ]:
pred_train[:10,0]

In [ ]:
y_test[:10]

In [ ]:
pred_test[:10,0]

In [ ]:
if scale:
    pred_train = scaler.inverse_transform(pred_train)
    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
    pred_test = scaler.inverse_transform(pred_test)
    y_test = scaler.inverse_transform(y_test.reshape(-1,1))
    


In [ ]:
y_train[:10],pred_train[:10,0]

In [ ]:
y_test[:10],pred_test[:10,0]

In [ ]:
# calculate root mean squared error
rsme_train = math.sqrt(mean_squared_error(y_train, pred_train[:,0]))
print('Train Score: %.2f RMSE' % (rsme_train))
rsme_test = math.sqrt(mean_squared_error(y_test, pred_test[:,0]))
print('Test Score: %.2f RMSE' % (rsme_test))

In [ ]:
print(len(ts_train), len(pred_train), len(y_train))
len(ts_test), len(pred_test), len(y_test) 

In [ ]:
# shift train predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_train_shifted = np.empty_like(ts_all)
print(pred_train_shifted.size)
pred_train_shifted[:, :] = np.nan
# train predictions start at position window_size + 1 (or window_size, if counting from 0)
pred_train_shifted[window_size : len(pred_train) + window_size, :] = pred_train
pred_train_shifted[:13]

In [ ]:
# shift test predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_test_shifted = np.empty_like(ts_all)
pred_test_shifted[:, :] = np.nan
pred_test_shifted[len(pred_train) + (window_size * 2) : len_overall + 1, :] = pred_test
pred_test_shifted[-13:]

In [ ]:
plt.plot(ts_all)
plt.plot(pred_train_shifted)
plt.plot(pred_test_shifted)
plt.show()

In [ ]:
plot_start = -100
plot_end = -1
plt.plot(ts_all[plot_start:plot_end])
plt.plot(pred_train_shifted[plot_start:plot_end])
plt.plot(pred_test_shifted[plot_start:plot_end])
plt.savefig(fig_name)
plt.show()


In [ ]:
def plot_results_multiple(predicted_data, true_data, prediction_window):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_window)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.savefig(fig_name)
    plt.show()

In [ ]:
if use_LSTM:
    plot_results_multiple(prediction_seqs_train, y_train, prediction_window)

In [ ]:
if use_LSTM:
    plot_results_multiple(prediction_seqs_test, y_test, prediction_window)